In [2]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo
import tensorflow as tf
from tensorflow.keras import regularizers
import numpy as np
import plotly.io
from sklearn.utils import shuffle
#import data, X = inputs, Y = outputs 
df = pd.read_excel('ANN_TRAIN_TEST_RAMON_T40_AIR_WATER.xlsx')
df = shuffle(df)#embaralha os dados

X = df.iloc[:,1:7].values
Y = df.iloc[:, [12,13]].values

#Parameter control
trainsize = 0.4
neurons = 4
activation_function = 'relu'
batchsize = 32
epochs = 250
hidden_layers_quantity = 2

#normalize data
from sklearn.preprocessing import StandardScaler
scalerx = StandardScaler()
scalery = StandardScaler()
X_scaled = scalerx.fit_transform(X)
Y_scaled = scalery.fit_transform(Y)

#split data  
#from sklearn.model_selection import train_test_split

#Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_scaled, Y_scaled, 
#                                                train_size = trainsize, 
#                                                random_state = 0)

#using K-fold to split the data
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10)


In [ ]:
#Building the ANN MODEL
#aplicar knn regressor, média e desvio padrão de r2 e msr
from sklearn.metrics import r2_score, mean_squared_error

resultados = []
fold  = 0

for train, test in kf.split(X_scaled):
    fold+=1

    Xtrain = X_scaled[train]
    Ytrain = Y_scaled[train]

  
    #gridsearchcv aplicar no xtrain ytrain
    Ytest = Y_scaled[test]
    Xtest = X_scaled[test]

In [3]:
#Building the ANN MODEL
from sklearn.metrics import r2_score, mean_squared_error

resultados = []
fold  = 0

for train, test in kf.split(X_scaled):
    fold+=1

    Xtrain = X_scaled[train]
    Ytrain = Y_scaled[train]

  
    #gridsearchcv aplicar no xtrain ytrain
    Ytest = Y_scaled[test]
    Xtest = X_scaled[test]

    # pega ros melhores parametros de resultado do gridsearchcv e treinar a rede com todos os dados de treino
    parametros = []

    for neurons in range(1, neurons+1):

        for hidden_layers_quantity in range (1, hidden_layers_quantity+1):

            ann = tf.keras.models.Sequential() #Initializethe ann model

            #creatin a sequence of hidden layers (adding according to the range)
            for _ in range(hidden_layers_quantity):
                ann.add(tf.keras.layers.Dense(units=neurons, 
                                            activation=activation_function))                        
                ann.add(tf.keras.layers.Dropout(0.2)) #DROPUT LAYER

            ann.add(tf.keras.layers.Dense(units = 2)) # OUTPUT LAYER

            #Compiling the ann, it adds the stochastic gradient descent method to control losses
            ann.compile(optimizer = 'adam', 
                        loss = 'mean_squared_error') #COMPILER (TRY DIFFERENT PARAMETERS)

            #TRAIN THE ANN MODEL
            ann.fit(Xtrain, Ytrain, 
                    batch_size = batchsize, 
                    epochs = epochs)
            Qpred = ann.predict(Xtest)
            
            rmse = mean_squared_error(Ytest, Qpred, squared = False)
            r2 = r2_score(Ytest, Qpred)
            resultados.append((fold, neurons, hidden_layers_quantity, rmse, r2))
            

        #salvar dados de loss de treino e teste durante a ann


Epoch 1/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.0560
Epoch 2/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - loss: 1.8343
Epoch 3/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step - loss: 1.4894
Epoch 4/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 477us/step - loss: 1.3136
Epoch 5/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - loss: 1.2073
Epoch 6/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step - loss: 1.0380
Epoch 7/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - loss: 1.0303
Epoch 8/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - loss: 0.8824
Epoch 9/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - loss: 0.8849
Epoch 10/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - loss: 0.7753
Epoch 11/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - loss: 0.7265
Epoch 12/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step - loss: 0.6924
Epoch 13/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - loss: 0.6944
Epoch 14/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step - loss: 0.7005
Epoch 15/250
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 51

In [ ]:
print(resultados)

In [ ]:
# RMSE: 0, R2:1
for result in resultados:
    print(f"FOLD{result[0]} -  Para {result[1]} neurônios e {result[2]} camadas, RMSE = {result[3]}, R2 = {result[4]}")


In [ ]:
#apply inverse transform on the Xtest and Ytest
Xtest_original = scalerx.inverse_transform(Xtest)
Ytest_original = scalery.inverse_transform(Ytest)

#predict function
Qpred = ann.predict(Xtest)
Qpred_original = scalery.inverse_transform(Qpred)

print(Qpred)

In [ ]:
graf_exp_water = go.Scatter(x=X[:, 5], 
                            y=Y[:, 0], 
                            mode='markers', 
                            name='EXPERIMENTAL_WATER', 
                            marker=dict(size=10, 
                                        color='purple', 
                                        symbol='pentagon'))

Qtest_water = go.Scatter(x=Xtest_original[:, 5], 
                         y=Ytest_original[:, 0], 
                         mode='markers', 
                         name='Qtest_WATER',
                         marker=dict(size=11, 
                                     color='green', 
                                     symbol='x', 
                                     line={'width': 2}))

Qpred_water = go.Scatter(x=Xtest_original[:, 5], 
                         y=Qpred_original[:, 0], 
                         mode='markers', 
                         name='Qpred_WATER', 
                         marker=dict(size=7, 
                                     color='red', 
                                     symbol='circle'))

data_water = [graf_exp_water, Qtest_water, Qpred_water]

layout_water = go.Layout(title='ANN_TRAIN_TEST_RAMON_T40_AIR_WATER', 
                         xaxis=dict(title='Vazão massica do ar (m_ar)'), 
                         yaxis=dict(title='Taxa de troca térmica Q_water'))

fig_water = go.Figure(data_water, layout_water)

pyo.plot(fig_water)

